### Metryki - Problem klasyfikacji:
1. [Import bibliotek](#a0)
2. [Metryki - Klasyfikacja binarna](#a1)
    1. [Accuracy - dokładność klasyfikacji](#a2)
    2. [Macierz konfuzji/pomyłek](#a3)
    3. [Krzywa ROC](#a4)
3. [Metryki - Klasyfikacja wieloklasowa](#a5)   
    1. [Accuracy - dokładność klasyfikacji](#a6)
    2. [Macierz konfuzji/pomyłek](#a7)
    3. [Raport klasyfikacji](#a8)

#### <a name='a0'></a> 1. Import bibliotek

In [0]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

#### <a name='a1'></a> 2. Metryki - Klasyfikacja binarna
#### <a name='a2'></a> 2.1 Accuracy - Dokładność klasyfikacji

####   $$Accuracy = \frac{correct\ predictions}{total\ predictions} * 100$$

In [0]:
y_true = np.array([1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1])
y_pred = np.array([0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1])

In [0]:
def accuracy(y_true, y_pred):
    correct = 0
    for idx, _ in enumerate(y_true):
        if y_true[idx] == y_pred[idx]:
            correct += 1
    return correct / len(y_true) * 100

In [0]:
accuracy(y_true, y_pred)

In [0]:
results = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
results = results.sort_values(by='y_true')
results['sample'] = range(1, len(y_true) + 1)
results

In [0]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_true'], mode='markers', name='y_true'), row=1, col=1)
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_pred'], mode='markers', name='y_pred'), row=2, col=1)
fig.update_layout(width=1000, height=800, title='Binary classifier predictions')
fig.show()

#### <a name='a3'></a> 2.2 Macierz konfuzji/pomyłek

In [0]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
cm

In [0]:
import plotly.figure_factory as ff

def plot_confusion_matrix(cm):
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_1', 'true_0'])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index), colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(width=500, height=500, title='Confusion Matrix', font_size=16)
    fig.show()

plot_confusion_matrix(cm)

In [0]:
cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
cm_df

In [0]:
tn, fp, fn, tp = cm.ravel()
print(f'TN - True Negative: {tn}')
print(f'FP - False Positive: {fp}')
print(f'FN - False Negative: {fn}')
print(f'TP - True Positive: {tp}')

In [0]:
# False Positive Rate - Type I error
fpr = fp / (fp + tn)
print(fpr)

In [0]:
# False Negative Rate - Type II error
fnr = fn / (fn + tp)
print(fnr)

In [0]:
# Precision - ile obserwacji przewidywanych jako pozytywne są w rzeczywistości pozytywne
precision = tp / (tp + fp)
print(precision)

In [0]:
# Recall - jak wiele obserwacji z wzystkich poytywnych sklasyfikowaliśmy jako pozytywne
recall = tp / (tp + fn)
print(recall)

#### <a name='a4'></a> 2.3 Krzywa ROC

In [0]:
# Krzywa ROC
from sklearn.metrics import roc_curve
fpr, tpr, tresh = roc_curve(y_true, y_pred, pos_label=1)

In [0]:
roc = pd.DataFrame({'fpr': fpr, 'tpr': tpr})
roc

In [0]:
def plot_roc_curve(y_true, y_pred):
    # Binary classification
    from sklearn.metrics import roc_curve
    fpr, tpr, tresh = roc_curve(y_true, y_pred, pos_label=1)

    fig = go.Figure(data=[go.Scatter(x=roc['fpr'], y=roc['tpr'], line_color='red', name='ROC Curve'),
                        go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line_dash='dash', line_color='navy')],
                    layout=go.Layout(xaxis_title='False Positive Rate',
                                    yaxis_title='True Positive Rate',
                                    title='ROC Curve',
                                    showlegend=False,
                                    width=800))
    fig.show()
plot_roc_curve(y_true, y_pred)

In [0]:
# AUC Score
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_true, y_pred)
roc_auc

In [0]:
# Gini coefficient
gini = roc_auc * 2 - 1
print(gini)

#### <a name='a5'></a> 3. Metryki - Klasyfikacja wieloklasowa

#### <a name='a6'></a> 3.1 Accuracy - Dokładność klasyfikacji

####   $$Accuracy = \frac{correct\ predictions}{total\ predictions} * 100$$

In [0]:
y_true = np.array([1, 0, 1, 2, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 2, 1, 1, 2, 2])
y_pred = np.array([0, 0, 1, 2, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 1, 2, 1, 2])

from sklearn.metrics import accuracy_score

accuracy_score(y_true, y_pred)

#### <a name='a7'></a> 3.2 Macierz konfuzji/pomyłek

In [0]:
cm = confusion_matrix(y_true, y_pred)
cm

In [0]:
def plot_confusion_matrix(cm):
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1', 'pred_2'], index=['true_2','true_1', 'true_0'])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index), colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(width=500, height=500, title='Confusion Matrix', font_size=16)
    fig.show()

plot_confusion_matrix(cm)

#### <a name='a8'></a> 3.3 Raport klasyfikacji

In [0]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))